## Implement Vanilla CNN

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
EPOCHS = 10

In [3]:
class ConvNet(tf.keras.Model):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.sequence = [] 
        self.sequence.append(tf.keras.layers.Conv2D(16, (3, 3), 
                                                    padding='same', 
                                                    activation='relu')) # 28x28x16 input_data
        self.sequence.append(tf.keras.layers.Conv2D(16, (3, 3),
                                                    padding='same', activation='relu'))
        self.sequence.append(tf.keras.layers.MaxPool2D((2, 2))) # 14x14x16
        self.sequence.append(tf.keras.layers.Conv2D(32, (3, 3), 
                                                    padding='same', 
                                                    activation='relu')) # 14x14x32 input_data
        self.sequence.append(tf.keras.layers.Conv2D(32, (3, 3),
                                                    padding='same', activation='relu'))
        self.sequence.append(tf.keras.layers.MaxPool2D((2, 2))) # 7x7x32
        self.sequence.append(tf.keras.layers.Conv2D(64, (3, 3), 
                                                    padding='same', 
                                                    activation='relu')) # 7x7x64 input_data
        self.sequence.append(tf.keras.layers.Conv2D(64, (3, 3),
                                                    padding='same', activation='relu'))
        self.sequence.append(tf.keras.layers.Flatten()) #-> 7*7*64=1568 Convert feature vector
        self.sequence.append(tf.keras.layers.Dense(2048, activation='relu'))
        self.sequence.append(tf.keras.layers.Dense(10, activation='softmax')) # 10 class classification
        
    def call(self, x, training=False, mask=None):
        for layer in self.sequence:
            x = layer(x)
        return x

## Training/test loop

In [4]:
@tf.function
def train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)
    
@tf.function
def test_step(model, images, labels, loss_object, optimizer, test_loss, test_accuracy):
    predictions = model(images)
    
    t_loss = loss_object(labels, predictions)
    test_loss(t_loss)
    test_accuracy(labels, predictions)

## Prepare Dataset

In [5]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
# Normalize
x_train, x_test = x_train / 255.0, x_test / 255.0

# x_train: (NUM_SAMPLE, 28, 28) -> (NUM_SAMPLE, 28, 28, 1) - Increase one dimension
x_train = x_train[..., tf.newaxis].astype(np.float32)
x_test = x_test[..., tf.newaxis].astype(np.float32)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

11493376/11490434 [==============================] - 1s 0us/step


## Define training parameter

In [6]:
# Create model
model = ConvNet()

# Define loss and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Define performance metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

## Training

In [7]:
for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)
        
    for test_images, test_labels in test_ds:
        test_step(model, test_images, test_labels, 
                  loss_object, optimizer, test_loss, test_accuracy)
        
    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch+1, 
                          train_loss.result(),
                          train_accuracy.result() * 100,
                          test_loss.result(),
                          test_accuracy.result() * 100))

Epoch 1, Loss: 0.12106101214885712, Accuracy: 96.24333190917969, Test Loss: 0.037823934108018875, Test Accuracy: 98.8699951171875
Epoch 2, Loss: 0.08268128335475922, Accuracy: 97.413330078125, Test Loss: 0.04028121381998062, Test Accuracy: 98.81999969482422
Epoch 3, Loss: 0.06608900427818298, Accuracy: 97.94611358642578, Test Loss: 0.038423892110586166, Test Accuracy: 98.87666320800781
Epoch 4, Loss: 0.05650171637535095, Accuracy: 98.24666595458984, Test Loss: 0.03696388006210327, Test Accuracy: 98.92500305175781
Epoch 5, Loss: 0.049776963889598846, Accuracy: 98.45433044433594, Test Loss: 0.03524621203541756, Test Accuracy: 98.97599792480469
Epoch 6, Loss: 0.04489044472575188, Accuracy: 98.6097183227539, Test Loss: 0.03338497504591942, Test Accuracy: 99.03333282470703
Epoch 7, Loss: 0.04102345183491707, Accuracy: 98.73428344726562, Test Loss: 0.034042734652757645, Test Accuracy: 99.0585708618164
Epoch 8, Loss: 0.038058966398239136, Accuracy: 98.82770538330078, Test Loss: 0.034017991274